In [ ]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
from dataset import CircularLearningRate

from tensorflow import config
assert config.list_physical_devices('GPU')

In [2]:
BATCH_SIZE = 32
MAXLEN = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)
KINDS_SIZE = len(dataset.KINDS)

def build_model(EMBED_DIM=110, UNITS=220):

    layer = input_text = keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, input_length=MAXLEN, mask_zero=True)(layer)
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(layer)
    layer = layers.add([layer,
            layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(layer)])
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
        layers.Softmax(name='K')(layers.LSTM(KINDS_SIZE)(layer))
    ]
    model = keras.Model(inputs=[input_text], outputs=outputs)

    jsmodel = keras.Model(inputs=[input_text], outputs=outputs[:-1])
    # keras.utils.plot_model(model, to_file='model.png')
    return model, jsmodel

model, jsmodel = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 64, 110)      4840        input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 64, 220)      582560      embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 64, 220)      776160      bidirectional[0][0]              
______________________________________________________________________________________________

In [3]:
def fit(train_validation, scheduler, verbose=1):
    train, valid = train_validation
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    callbacks = []
    if isinstance(scheduler, CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    #     train.kind = np.ones((len(train), 1))
    #     valid.kind = np.ones((len(valid), 1))
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin, 'C': train.normalized, 'K': train.kind }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin, 'C': valid.normalized, 'K': valid.kind }
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


In [4]:
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [ ]:
data_rabanit = load_data(['rabanit'])

In [ ]:
data_pre_modern = load_data(['pre_modern'])

In [5]:
data_modern = load_data(validation=0.2, source=['modern'])

(13856,)
(13856, 64)


In [8]:
data_all = load_data(validation=0.1, source=['rabanit', 'pre_modern', 'modern'])

(154697,)
(154697, 64)


In [ ]:
model.load_weights('./checkpoints/uninit')
history = fit(data_rabanit, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))
model.save_weights('./checkpoints/rabanit')

In [ ]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, scheduler=CircularLearningRate(20e-4, 40e-4, 0.1e-4))
model.save_weights('./checkpoints/pre_modern')

In [6]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, scheduler=CircularLearningRate(6e-3, 6e-3, 0.5e-3))
# history = fit(data_modern, CircularLearningRate(1e-3, 2e-3, 0.5e-3))
model.save_weights('./checkpoints/modern')

Train on 13856 samples, validate on 3464 samples
13856/13856 [==============================] - 29s 2ms/sample - loss: 1.1777 - N_loss: 0.5031 - D_loss: 0.1207 - S_loss: 0.0247 - K_loss: 0.5293 - N_accuracy: 0.8198 - D_accuracy: 0.9525 - S_accuracy: 0.9941 - K_accuracy: 0.9978 - val_loss: 0.8426 - val_N_loss: 0.2496 - val_D_loss: 0.0695 - val_S_loss: 0.0068 - val_K_loss: 0.5169 - val_N_accuracy: 0.9101 - val_D_accuracy: 0.9720 - val_S_accuracy: 0.9981 - val_K_accuracy: 1.0000


In [ ]:
model.load_weights('./checkpoints/uninit')
history = fit(data_all, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))
# history = fit(data_modern, CircularLearningRate(1e-3, 2e-3, 0.5e-3))
model.save_weights('./checkpoints/all')

Train on 154697 samples, validate on 17189 samples
154688/154697 [============================>.] - ETA: 0s - loss: 1.3124 - N_loss: 0.2792 - D_loss: 0.0742 - S_loss: 0.0086 - K_loss: 0.9505 - N_accuracy: 0.9007 - D_accuracy: 0.9708 - S_accuracy: 0.9977 - K_accuracy: 0.7006

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
jsmodel.summary()

In [ ]:
import tensorflowjs as tfjs
jsmodel.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(jsmodel, '.')

In [7]:
model.load_weights('./checkpoints/modern')

def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin, actual_kind] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2]),  dataset.from_categorical(prediction[3])]
    [expected_niqqud, expected_dagesh, expected_sin, expected_kind] = [data.niqqud[s], data.dagesh[s], data.sin[s], data.kind[s]]
    actual = dataset.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a, actual_kind[i])
        print('רצוי: ', e, expected_kind[i])

print_predictions(data_modern[1], 2)

מצוי:  שֶׁל הַפְּצָצָה אֲוִוירִית רַחֶבַת הַיֶּקֵּף עַל יַפָּן. הַפִּיקּוּד הַצְּבָאִי הָאָמֵרִיקָאִי הֵחֵל  4
רצוי:  שֶׁל הַפְצָצָה אֲוִוירִית רַחֲבַת הֶיקֵּף עַל יַפָּן. הַפִּיקּוּד הַצְּבָאִי הָאָמֵרִיקָאִי הֵחֵל  4
מצוי:  שֶׁלָּנוּ וְהִצְּטוּפְּפוּ לְאוֹרֶךְ הַתּוֹר בְּחַדְרֵי הַהִתְעַמְּלוּת שֶׁל הַתִּיכּוּן, אוֹ רָאוּ  4
רצוי:  שֶׁלָּנוּ וְהִצְטוֹפְפוּ לְאוֹרֶךְ הַתּוֹר בְּחַדְרֵי הַהִתְעַמְּלוּת שֶׁל הַתִּיכוֹן, אוֹ רָאוּ  4
מצוי:  פְּעוּלָה עִם הָאֲצוּלָּה הַגְּבוֹהָה עַל מְנַת לְחַזֵק אֶת מַעֶמְדָם מוּל הֶעָרִים אוֹ לְשַׁתֵּף  4
רצוי:  פְּעוּלָּה עִם הָאֲצוּלָּה הַגְּבוֹהָה עַל מְנַת לְחַזֵּק אֶת מַעֲמָדָם מוּל הֶעָרִים אוֹ לְשַׁתֵּף  4
מצוי:  בָּהּ, אֶגַב, יֵשׁ לִנִיּוּז קוֹרֵפ מִנִיּוֹת - עָזֶר לָנוּ לַאֲתֵר דִּיוּּוּחַ בּוֹדֵד. הוּא  4
רצוי:  בָּהּ, אַגַּב, יֵשׁ לִנְיוּז קוֹרְפּ מְנָיוֹת - עָזַר לָנוּ לְאַתֵּר דִּיוּוּחַ בּוֹדֵד. הוּא  4
מצוי:  הַקּוֹנְטִינְנְטְלִי מִיְיחֵס לְתַקָדִים חֲשִׁיבוּת פְּחוּתָה, וְהַמָּקוֹר הָרִאשׁוֹנִי בּוֹ מוֹצֵא  4
רצוי:  הַקּוֹנְטִינֶנְטָלִי 

In [ ]:
shutil.rmtree(os.sep.join([tempfile.gettempdir(), '.tensorboard-info']), ignore_errors=True)
shutil.rmtree('logs', ignore_errors=True)
os.makedirs('logs')
# %tensorboard --logdir logs

In [ ]:
print(data_modern[1].text[0])
print(data_modern[1].text[1])